In [21]:
import numpy as np 
import pandas as pd 
%matplotlib inline
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import KFold 
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import cvxopt

# Load Data

In [22]:
Y_train_data = pd.read_csv('Ytr.csv', index_col=0)
Y_train_data.head()

,Bound
Id,
0,1
1,0
2,1
3,0
4,1


In [23]:
X_train_data = pd.read_csv('Xtr.csv', index_col=0)  #header=None, sep=' '
#X_train.drop(columns='Id', inplace=True)
X_test_data = pd.read_csv('Xte.csv', index_col=0)     #header=None, sep=' '
#X_test.drop(columns='Id', inplace=True)

# Data Preprocessing

In [24]:
WORD_LENTGH = 7
base = {'A':0, 'C':1, 'T':2, 'G':3}
index_size = 4**WORD_LENTGH


def word_to_index(word):
    index = 0
    for i, carac in enumerate(word):
        index += base[carac] * (4 ** i)
    return index

def encode_sequence(seq, word_len=WORD_LENTGH):
    encoded = np.zeros(index_size)
    for i in range(len(seq) - word_len):
        word = seq[i:(i+word_len)]
        index = word_to_index(word)
        encoded[index] += 1
    return encoded

Xtr_encoded = X_train_data.seq.apply(encode_sequence)
Xtr_encoded = np.stack(Xtr_encoded)

print('Xtr_encoded', Xtr_encoded.shape)
print('max: {}, mean: {}'.format(Xtr_encoded.max(), Xtr_encoded.mean()))
print('non zeros: {}, zeros: {},'.format(np.sum(Xtr_encoded==0), np.sum(Xtr_encoded!=0)))
print('ratio: {}'.format(np.sum(Xtr_encoded!=0)/(Xtr_encoded.shape[0]*Xtr_encoded.shape[1])))

Xtr_encoded (2000, 16384)
max: 15.0, mean: 0.0057373046875
non zeros: 32583265, zeros: 184735,
ratio: 0.005637664794921875


In [25]:
Xte_encoded = X_test_data.seq.apply(encode_sequence)
Xte_encoded = np.stack(Xte_encoded)

print('Xte_encoded', Xte_encoded.shape)
print('max: {}, mean: {}'.format(Xte_encoded.max(), Xte_encoded.mean()))
print('non zeros: {}, zeros: {},'.format(np.sum(Xte_encoded==0), np.sum(Xte_encoded!=0)))
print('ratio: {}'.format(np.sum(Xte_encoded!=0)/(Xte_encoded.shape[0]*Xte_encoded.shape[1])))

Xte_encoded (1000, 16384)
max: 21.0, mean: 0.0057373046875
non zeros: 16291596, zeros: 92404,
ratio: 0.005639892578125


# Split Data : Train/Validation

In [26]:
from sklearn.model_selection import train_test_split

X = Xtr_encoded
y = Y_train_data.Bound.values * 2 - 1

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Define Kernel

In [55]:
#SVM are using linear_kernel1 and rbf_kernel2

def rbf_kernel1(gamma, **kwargs):
    def f(x1, x2):
        distance = np.linalg.norm(x1-x2)**2
        return np.exp(-gamma*distance)
    return f

def linear_kernel1(**kwargs):
    def f(x1, x2):
        return x1 @ x2.T
    return f

def rbf_kernel(X1, X2, sigma=10):
    
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis = -1)
    X1_norm = np.sum(X1 ** 2, axis = -1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def linear_kernel(X1, X2):

    return X1.dot(X2.T)

# Define model

# Ridge Regression

In [52]:
class RidgeClassifier():
    def __init__(self, lambd, sigma):     #: #, sigma):   # When using the linear kernel, sigma should be commented
        self.lambd = lambd
        self.sigma = sigma

    def fit(self, X, y):
        self.X_train = X

        n,p = X.shape
        I = np.eye(n)
        A = rbf_kernel(X, self.X_train, sigma= self.sigma) + lambd*n*I
        #A = linear_kernel(X, self.X_train) + lambd*n*I
        self.alpha = np.linalg.solve(A, y)    

    def predict(self, X):    
        output = rbf_kernel(X, self.X_train, sigma= self.sigma) @ self.alpha
        #output = linear_kernel(X, self.X_train) @ self.alpha
        pred = np.sign(output)
        return pred

    def score(self, X, y):
        pred = self.predict(X)
        accuracy = np.sum(pred == y) /len(y)
        return accuracy

# SVM FROM SCRATCH

In [56]:
class SupportVectorMachine(object):
    def __init__(self, C=3, kernel= linear_kernel1, power=4, gamma= 2, coef= 2):
        self.C = C
        #self.kernel = kernel
        self.power = power
        self.gamma = gamma
        self.coef = coef
        self.lagr_multipliers = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.intercept = None
        self.kernel = kernel(power = self.power, gamma= self.gamma, coef = self.coef)
        
    def fit(self, X, y):
        n_samples, n_features = np.shape(X)
        #Set gamma to 1/n_features by default
        if not self.gamma:
            self.gamma = 1/n_features
        #Initialize kernel method with parameters
        #self.kernel = self.kernel(power = self.power, gamma= self.gamma, coef = self.coef)
        #Calculate kernel matrix
        kernel_matrix = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range (n_samples):
                kernel_matrix[i, j] = self.kernel(X[i], X[j])

        # Define the quadratic optimization problem
        P = cvxopt.matrix(np.outer(y, y) * kernel_matrix, tc='d')
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1, n_samples), tc='d')
        b = cvxopt.matrix(0, tc='d')

        if not self.C: #if its empty
            G = cvxopt.matrix(np.identity(n_samples) * -1)
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            G_max = np.identity(n_samples) * -1
            G_min = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((G_max, G_min)))
            h_max = cvxopt.matrix(np.zeros(n_samples))
            h_min = cvxopt.matrix(np.ones(n_samples) * self.C)
            h = cvxopt.matrix(np.vstack((h_max, h_min)))

        # Solve the quadratic optimization problem using cvxopt
        minimization = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers
        lagr_mult = np.ravel(minimization['x'])

        # Extract support vectors
        # Get indexes of non-zero lagr. multipiers
        idx = lagr_mult > 1e-11
        # Get the corresponding lagr. multipliers
        self.lagr_multipliers = lagr_mult[idx]
        # Get the samples that will act as support vectors
        self.support_vectors = X[idx]
        # Get the corresponding labels
        self.support_vector_labels = y[idx]

        # Calculate intercept with first support vector
        self.intercept = self.support_vector_labels[0]
        for i in range(len(self.lagr_multipliers)):
            self.intercept -= self.lagr_multipliers[i] * self.support_vector_labels[i] * self.kernel(self.support_vectors[i], self.support_vectors[0])

    def predict(self, X):
        y_pred = []
        # Iterate through list of samples and make predictions
        for sample in X:
            prediction = 0
            # Determine the label of the sample by the support vectors
            for i in range(len(self.lagr_multipliers)):
                prediction += self.lagr_multipliers[i] * self.support_vector_labels[i] * self.kernel(self.support_vectors[i], sample)
            prediction += self.intercept
            y_pred.append(np.sign(prediction))
            
        return np.array(y_pred)
    
    def score(self, X, y):
        pred = self.predict(X)
        return np.mean(pred==y)

In [58]:
print ("-- SVM Classifier --")



#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf = SupportVectorMachine(C=2.07 , kernel=linear_kernel1, gamma = 4.78e-03)
clf.fit(X_train, y_train)

train_accuracy = clf.score(X_train, y_train)
print ("Train Accuracy scracht :", train_accuracy)

valid_accuracy = clf.score(X_val, y_val)
print ("Valid Accuracy scracht :", valid_accuracy)



-- SVM Classifier --
     pcost       dcost       gap    pres   dres
 0:  2.9475e+01 -4.3869e+03  1e+04  8e-01  5e-15
 1:  5.8770e+01 -1.1343e+03  2e+03  8e-02  6e-15
 2:  2.9046e+01 -2.5422e+02  3e+02  8e-03  6e-15
 3: -4.3675e+00 -3.3102e+01  3e+01  2e-04  4e-15
 4: -9.7997e+00 -1.2897e+01  3e+00  3e-06  2e-15
 5: -1.0099e+01 -1.0581e+01  5e-01  5e-07  2e-15
 6: -1.0148e+01 -1.0206e+01  6e-02  3e-08  2e-15
 7: -1.0155e+01 -1.0158e+01  3e-03  1e-09  2e-15
 8: -1.0156e+01 -1.0156e+01  9e-05  2e-11  2e-15
 9: -1.0156e+01 -1.0156e+01  3e-06  3e-13  2e-15
Optimal solution found.
Train Accuracy scracht : 1.0
Valid Accuracy scracht : 0.66


# Train model

In [41]:
lambd=3.26e-01
sigma =11.61
model = RidgeClassifier(lambd=lambd ,sigma=sigma) #sigma=sigma)

model.fit(X_train, y_train)
#pred, final_scores = logistic_regression(X_train,  (y_train+1)/2, num_steps=100, learning_rate=0.05)
train_acc = model.score(X_train, y_train)
valid_acc = model.score(X_val, y_val)

print(f' Train_accuracy = {train_acc}, Val_accuracy = {valid_acc}')

#(final_scores).sum()

 Train_accuracy = 0.7955555555555556, Val_accuracy = 0.655


## Define Cross Validation

In [32]:
# generate random range of value 
def rand_range(begin, end):
    return (end-begin)*np.random.rand() + begin

In [33]:
from sklearn.model_selection import KFold

def cross_validation(model, X, y, cv=5):
    kf = KFold(n_splits=cv, shuffle=True)
    kf.get_n_splits(X)

    scores = []
    train_scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        acc_train = model.score(X_train, y_train)
        accuracy  = model.score(X_test, y_test)
        
        #model.train(X_test, y_test)
        #accuracy = model.score(X_train, y_train)
        
        train_scores.append(acc_train)
        scores.append(accuracy)
    
    train_scores = np.array(train_scores)
    scores = np.array(scores)
    return scores, train_scores

# scores = cross_validation(X_train, y_train)
# scores
# mean_acc = scores.mean()

# Random Search with CV

Cross validation 0n 5 folders

In [22]:
print_saved = '' #run this to erase or init the history
from IPython.display import clear_output
    
def prints(msg):
    global print_saved
    print_saved = msg + '\n' + print_saved
    clear_output(wait=True)
    print(print_saved)

In [23]:
# prints(' i | C     | gamma  | accuracy | mean_train | scores for all folds')
# prints('---|-------|--------|----------|------------|---------')

# prints(' i |gamma   | accuracy | mean_train | scores for all folds')
# prints('---|--------|----------|------------|---------')

prints(' i | C     | gamma  | accuracy | mean_train | scores for all folds')
prints('---|-------|--------|----------|------------|---------')
#.........,..........,.......,...........,....................
for i in range(10):
    #C     = rand_range(0.1, 2)
    #gamma = 10**rand_range(-3.5, -2.5)
    #sigma = rand_range(2, 10)
    lambd= 10**rand_range(-2, 1)
    
    #model = SupportVectorMachine(C=3, kernel= rbf_kernel, gamma= gamma, coef= 2)
    model = RidgeClassifier(lambd) #, sigma)
    
    #scores, train_scores = cross_validation(model, X_train, y_train, cv=3)
    scores, train_scores = cross_validation(model, X, y, cv=5)
    mean_acc = scores.mean()
    mean_train = train_scores.mean()
    
    #prints(f'{i:2d} | {C:2.2f} | {gamma:2.2e}  | {100*mean_acc:.2f}%    | {100*mean_train:.2f}%   | {scores}')     # for SVM with RBF kernel
    #prints(f'{i:2d} | {C:2.2f} | {gamma:2.2e}  | {100*mean_acc:.2f}%    | {100*mean_train:.2f}%   | {scores}')     # for SVM with linear kernel
    prints(f'{i:2d} | {lamda: 2.2e} | {sigma:2.2e}  | {100*mean_acc:.2f}%    | {100*mean_train:.2f}%   | {scores}') #for RidgeRegression with RBF kernel
    #prints(f'{i:2d} | {lambd:2.2e}  | {100*mean_acc:.2f}%    | {100*mean_train:.2f}%   | {scores}')                # for RidgeRegression with linear kernel

 9 | 3.26e-01  | 67.45%    | 95.79%   | [0.6575 0.685  0.685  0.6525 0.6925]
 8 | 3.26e-01  | 68.25%    | 95.74%   | [0.685  0.695  0.6875 0.6775 0.6675]
 7 | 3.26e-01  | 68.60%    | 95.77%   | [0.68   0.725  0.6775 0.6425 0.705 ]
 6 | 3.26e-01  | 67.05%    | 95.84%   | [0.63   0.68   0.685  0.6675 0.69  ]
 5 | 3.26e-01  | 68.90%    | 95.76%   | [0.69   0.68   0.7175 0.66   0.6975]
 4 | 3.26e-01  | 66.70%    | 95.76%   | [0.6525 0.65   0.7    0.635  0.6975]
 3 | 3.26e-01  | 67.15%    | 95.89%   | [0.6875 0.63   0.66   0.685  0.695 ]
 2 | 3.26e-01  | 67.45%    | 95.84%   | [0.63   0.6875 0.6675 0.675  0.7125]
 1 | 3.26e-01  | 68.20%    | 95.80%   | [0.685  0.6725 0.675  0.6875 0.69  ]
 0 | 3.26e-01  | 67.45%    | 95.80%   | [0.675  0.66   0.675  0.66   0.7025]
---|-------|--------|----------|------------|---------
 i | C     | gamma  | accuracy | mean_train | scores for all folds



# Evaluation train/valid

In [24]:
def accuracy_score(y_test, y_pred):
    correct_pred = np.sum(y_test == y_pred)
    total = len(y_test)
    accuracy = correct_pred/ total
    return accuracy

In [36]:
sigma = 9
lambd = 3.26e-01
#C=2.07   parameter of svm                    
#gamma = 4.78e-03   svm              

#model = RidgeClassifier(lambd)# with linear kernel
#model = RidgeClassifier(lambd, sigma)# with linear kernel
alpha = model.fit(X_train, y_train)

#model = SupportVectorMachine(#C=C) #SVM with rbf_kernel
#model = SupportVectorMachine(#C=C, gamma=gamma) #SVM with rbf_kernel


model.fit(X_train, y_train)

accuracy     = model.score(X_train, y_train)
val_accuracy = model.score(X_val, y_val)

print(f'Train {accuracy:.2f} | Valid {val_accuracy:.2f}')

Train 0.76 | Valid 0.60


In [26]:
scores, fit_scores = cross_validation(model, X, y, cv=5)
scores

array([0.715 , 0.6875, 0.6425, 0.68  , 0.6625])

In [27]:
scores.mean()

0.6775

# Prepare Submission

In [28]:
my_prediction = model.predict(Xte_encoded)

my_prediction[my_prediction== -1] = 0
my_prediction= my_prediction.astype(int)

submission = pd.DataFrame({'Bound': my_prediction})
submission.index.name = 'Id'
submission.head()

,Bound
Id,
0,1
1,1
2,0
3,0
4,1


In [29]:
# statistics on the prediction
zeros = (my_prediction==0).sum()
ones  = (my_prediction==1).sum()
print(f'zeros: {zeros} | ones: {ones}')


zeros: 429 | ones: 571


In [30]:
submission.to_csv('submission_71acc.csv')

In [31]:
#!kaggle competitions submit -c kernel-methods-ammi-2020 -f 'submission_71acc.csv' -m 'length=6 model'